#Daffa Rizki Rizaly - 2301931251

In [1]:
!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [2]:
#Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNNCell, RNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
#Import dataset
def load_data():
    df = pd.read_csv('USD_INR.csv',sep =',', index_col='Date')
    df.index = pd.to_datetime(df.index)
    
    return df

dataset = load_data()

#Preprocessing

In [5]:
#Initialize layers
layers = {
    'input': 5, #5 as there is 5 features
    'output' : 5 #5 as there is 5 output
}

In [6]:
#Initialize variables
time_steps = 20
batch_size = 2
learning_rate = 0.005
epoch = 5000

In [7]:
#Spliting dataset
split = int(len(dataset) * 0.8)
training_dataset = dataset[ :split]
testing_dataset = dataset[split: ]

In [8]:
# preprocess
scaler = MinMaxScaler(feature_range=(0, 1)).fit(dataset)
normal_train = scaler.transform(training_dataset)
normal_test = scaler.transform(testing_dataset)

In [9]:
# membuat model
context_count = 100
cell = tf.nn.rnn_cell.BasicLSTMCell(context_count, activation=tf.nn.relu)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [10]:
# passing informasi ke cell berikutnya
cell = tf.contrib.rnn.OutputProjectionWrapper(cell, output_size=layers['output'], activation=tf.nn.relu)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [11]:
x = tf.placeholder(tf.float32, [None, time_steps, layers['input']])
y = tf.placeholder(tf.float32, [None, time_steps, layers['output']])

In [12]:
output, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
loss = tf.reduce_mean(0.5 * (y - output) ** 2)

In [14]:
# Adam = (Adaptive Gradient Algorithm) & (Root Mean Square Propagation)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
def next_batch(dataset, batch_size, time_steps):
    x_batch = np.zeros(shape=(batch_size, time_steps, layers['input']))
    y_batch = np.zeros(shape=(batch_size, time_steps, layers['output']))
    
    for i in range(batch_size):
        start = np.random.randint(0, len(dataset) - time_steps)
        x_batch[i] = dataset[start : start + time_steps]
        y_batch[i] = dataset[start + 1: start + time_steps + 1]
        
    return x_batch, y_batch

saver = tf.train.Saver()


In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(1, epoch+1):
        x_batch, y_batch = next_batch(normal_train, batch_size, time_steps)
        
        feed_train = {
            x: x_batch,
            y: y_batch
        }
        
        sess.run(train, feed_dict=feed_train)
        if i % 200 == 0:
            print('Epoch: ', i, 'Loss: ', end="")
            print(sess.run(loss, feed_dict=feed_train))
    
    saver.save(sess, 'rnn-model/model.ckpt')

Epoch:  200 Loss: 0.00040455553
Epoch:  400 Loss: 0.00010191919
Epoch:  600 Loss: 0.00014336657
Epoch:  800 Loss: 4.701052e-05
Epoch:  1000 Loss: 8.407846e-05
Epoch:  1200 Loss: 7.6046046e-05
Epoch:  1400 Loss: 4.562468e-05
Epoch:  1600 Loss: 8.369127e-05
Epoch:  1800 Loss: 3.0778032e-05
Epoch:  2000 Loss: 1.1160993e-05
Epoch:  2200 Loss: 2.0730933e-05
Epoch:  2400 Loss: 1.8318447e-05
Epoch:  2600 Loss: 0.00018420882
Epoch:  2800 Loss: 8.433099e-05
Epoch:  3000 Loss: 8.987006e-05
Epoch:  3200 Loss: 2.6353966e-05
Epoch:  3400 Loss: 1.5651221e-05
Epoch:  3600 Loss: 3.3767614e-05
Epoch:  3800 Loss: 1.6582119e-06
Epoch:  4000 Loss: 1.9881265e-05
Epoch:  4200 Loss: 0.00013810383
Epoch:  4400 Loss: 4.434724e-05
Epoch:  4600 Loss: 1.915559e-05
Epoch:  4800 Loss: 3.4095443e-05
Epoch:  5000 Loss: 1.6467784e-05


In [17]:
# Testing
with tf.Session() as sess:
    seed_data = list(normal_test)
    saver.restore(sess, 'rnn-model/model.ckpt')
    
    for i in range(len(testing_dataset)):
        x_batch = np.array(seed_data[-time_steps: ]).reshape(5, time_steps, layers['input'])
        
        y_pred = sess.run(output, feed_dict={x: x_batch})
        # print(y_pred)
        
        seed_data.append(y_pred[0, -1, 0])
        # print(seed_data)

INFO:tensorflow:Restoring parameters from rnn-model/model.ckpt


ValueError: ignored